In [3]:
# Import Required Python Base Packages
import sys
import unittest
import numpy as np
import random
import collections
import os
import time as tp
import glob

# Create a Safety Mechanism for Not Having MatPlotLib Installed
try:
    %matplotlib inline
    import matplotlib.pyplot as plt
    from matplotlib.pyplot import *
    from mpl_toolkits.mplot3d import Axes3D
    HAS_MATPLOTLIB = True
    
    matplotlib.rcParams['figure.figsize'] = (16, 6)
    matplotlib.rcParams['font.size'] = (14)
except ImportError:
    HAS_MATPLOTLIB = False

# Import the Amuse Base Packages
from amuse import datamodel
from amuse.units import nbody_system
from amuse.units import units
from amuse.units import constants
from amuse.datamodel import particle_attributes
from amuse.io import *
from amuse.lab import *

# Import the Amuse Stellar Packages
from amuse.ic.kingmodel import new_king_model
from amuse.ic.salpeter import new_salpeter_mass_distribution_nbody

# Import the Amuse Gravity & Close-Encounter Packages
from amuse.community.ph4.interface import ph4 as grav
from amuse.community.smalln.interface import SmallN
from amuse.community.kepler.interface import Kepler
from amuse.couple import multiples

# Importing cPickle/Pickle
try:
   import cPickle as pickle
except:
   import pickle

from tycho import read, util

In [4]:
# Import Encounter file
file_prefix = "Bullhead"
file_directory = "/home/draco/jthornton/Bullhead3/"
file_end = "_encounters.pkl"

encounterInfoReload = None
encounter_file = open(file_directory+file_prefix+file_end, "rb")
encounterInfoReload = pickle.load(encounter_file)
encounter_file.close()

In [22]:
def get_component_binary_elements(comp1, comp2, kep, kep2, peri = 0):

    mass = comp1.mass + comp2.mass
    pos = comp2.position - comp1.position
    vel = comp2.velocity - comp1.velocity
    
    if comp1.mass<comp2.mass:
        Mp = comp1.mass
        Ms = comp2.mass
        planet = comp1
        hoststar = comp2
    else:
        Mp = comp2.mass
        Ms = comp1.mass
        planet = comp2
        hoststar = comp1
        
    # For Dopplershift
    rel_pos = planet.position - hoststar.position
    rel_vel = planet.velocity - hoststar.velocity
    kep2.initialize_from_dyn(mass, rel_pos[0], rel_pos[1], rel_pos[2], \
    rel_vel[0], rel_vel[1], rel_vel[2])
    
    kep.initialize_from_dyn(mass, pos[0], pos[1], pos[2],
                            vel[0], vel[1], vel[2])
    a,e = kep.get_elements()
    r = kep.get_separation()
    E,J = kep.get_integrals()           # per unit reduced mass, note
    if peri:
        M,th = kep.get_angles()
        if M < 0:
            kep.advance_to_periastron()
        else:
            kep.return_to_periastron()
    t = kep.get_time()
    P = kep.get_period()

    # Calculating Angular Momentum Vector, h = r x v
    h = np.cross(rel_pos, rel_vel)
    # Calculating the Inclination in Radians 
    # https://en.wikibooks.org/wiki/Astrodynamics/Classical_Orbit_Elements#Inclin$
    I = np.arccos(h[2]/np.sqrt(h.dot(h)))
    K = (2*np.pi/P)*(Mp*np.sin(I)/mass)*(a/np.sqrt(1-e**2))
    
    return mass,a,e,r,E,t,P,K

In [23]:
Amplitude = []
Period = []

In [8]:
from collections import defaultdict

encounterInformation = defaultdict(list)
for key in encounterInfoReload:
    encounterInformation[key] = []
Mass = []
semimajor = []
eccentricity = []
Energy = []
Time = []
Period = []
Amplitude = []
initial_dir = file_directory+"/InitialState"
master_set, ic_array, converter = read_initial_state(initial_dir, file_prefix)
kep = Kepler(unit_converter=converter, redirection="none")
kep.initialize_code()
kep2 = Kepler(unit_converter=converter, redirection="none")
kep2.initialize_code()

In [44]:
from tycho import create

num_stars = 1
num_planets = 0
IBF = 1
seed = 0
stars_SI, converter = create.king_cluster(num_stars, num_binaries=IBF, seed=seed)


In [52]:
num_star=1
num_planet=1
ibf=0

stars_SI2, converter2 = create.king_cluster(num_star, num_binaries=ibf, seed=seed)
systems_SI = create.planetary_systems(stars_SI2, converter2, num_planet, 'test_planets', Jupiter=True)
MasterSet = datamodel.Particles()
MasterSet.add_particles(stars_SI2)
MasterSet.add_particles(systems_SI)

In [66]:
try:
    kep2.stop()
    kep.stop()
except:
    pass
kep = Kepler(unit_converter=converter, redirection="none")
kep.initialize_code()

kep2 = Kepler(unit_converter=converter2, redirection="none")
kep2.initialize_code()

In [53]:
print MasterSet

                 key    host_star           id         mass       radius         type           vx           vy           vz            x            y            z
                   -         none         none           kg            m         none    m * s**-1    m * s**-1    m * s**-1            m            m            m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
  275342694097063007            0            1    9.039e+28    6.798e+12         star    0.000e+00    0.000e+00    0.000e+00    0.000e+00    0.000e+00    0.000e+00
 1200733817644779700            1      5000000    1.899e+27    4.551e+08       planet    1.928e+03    3.144e+03    1.184e+03    1.029e+12   -1.077e+11    3.876e+11
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========


In [45]:
print stars_SI

                 key           id         mass       radius         type           vx           vy           vz            x            y            z
                   -         none           kg           AU         none    m * s**-1    m * s**-1    m * s**-1            m            m            m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
18033081871020678442            2    9.424e+29    4.738e+02         star    2.848e+02    1.436e+03    3.322e+02    3.226e+12   -1.082e+12    7.465e+11
 6156527733378841055            3    5.385e+29    2.707e+02         star   -4.983e+02   -2.512e+03   -5.814e+02   -5.646e+12    1.894e+12   -1.306e+12
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========


In [9]:
mass,a,e,r,E,t,P = get_component_binary_elements(stars_SI[0], stars_SI[1],kep)

NameError: name 'stars_SI' is not defined

In [68]:
mass,a,e,r,E,t,P = get_component_binary_elements(MasterSet[0],MasterSet[1],kep2)

5105531886.96 s


In [71]:
print mass
print stars_SI[0].mass+stars_SI[1].mass
print a
print e
print r
print E
print P.as_quantity_in(units.yr)

1.4808462008e+30 kg
1.4808462008e+30 kg
2.74724342534e+13 m
0.653687862156
9.5804601721e+12 m
-1798818.06066 m**2 * s**-2
2883.856872 yr


In [26]:
# Currently outputting 0 for all times in the oneset I've tried

# For loop to go through all keys in the dictionary
for key in encounterInfoReload:
    
    # try loop to cover for empty spaces in the dictionary for shorter runs
    i=0
    while i < len(encounterInfoReload[key])-2:

        # Method from Tyler's code
        mass,a,e,r,E,t,P,K = get_component_binary_elements(encounterInfoReload[key][i][0],encounterInfoReload[key][i+1][1],kep,kep2)
        # i +2 here to loop through the dictionary correctly (function takes 2 elements at a time)
        i+=2
        print K

        # Store the information in lists in the loop then append afterwards
        Mass.append(mass)
        semimajor.append(a)
        eccentricity.append(e)
        Energy.append(E)
        Time.append(t)
        Period.append(P)
        Amplitude.append(K)

    # Store information in dictionary
    encounterInformation[key].append(Mass)
    encounterInformation[key].append(semimajor)
    encounterInformation[key].append(eccentricity)
    encounterInformation[key].append(Energy)
    encounterInformation[key].append(Time)
    encounterInformation[key].append(Period)
    encounterInformation[key].append(Amplitude)

    # Clean the lists
    Mass = []
    semimajor = []
    eccentricity = []
    Energy = []
    Time = []
    Period = []
    Amplitude = []
   

120.636730974 m * s**-1
114.695922132 m * s**-1
112.326472691 m * s**-1
110.458282512 m * s**-1
109.235004565 m * s**-1
108.096024661 m * s**-1
107.182518282 m * s**-1
108.125589683 m * s**-1
370.314983102 m * s**-1
14.3279055487 m * s**-1
12.7877170994 m * s**-1
321.345002414 m * s**-1
14.3279055487 m * s**-1
12.7877170994 m * s**-1
nan m * s**-1
50.3982356702 m * s**-1
76659.1216929 m * s**-1
nan m * s**-1
nan m * s**-1
24278.4304756 m * s**-1
nan m * s**-1
117.882678943 m * s**-1
123.204149321 m * s**-1
108.081446635 m * s**-1
107.314201931 m * s**-1
118.41637005 m * s**-1
5498.34805705 m * s**-1
101.770872948 m * s**-1
202.109545272 m * s**-1
123.204149321 m * s**-1
108.081446635 m * s**-1
107.314201931 m * s**-1
118.41637005 m * s**-1
5498.34805705 m * s**-1
101.770872948 m * s**-1
nan m * s**-1
161.023034617 m * s**-1
20491.5536833 m * s**-1
nan m * s**-1
nan m * s**-1
104.006582802 m * s**-1
24278.4304756 m * s**-1
nan m * s**-1
nan m * s**-1
nan m * s**-1


/home/draco/jthornton/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:42: RuntimeWarning: invalid value encountered in sqrt


1773.94676491 m * s**-1
781.163912805 m * s**-1
467.00753139 m * s**-1
467.593285323 m * s**-1
466.286431345 m * s**-1
464.259392568 m * s**-1
463.194656914 m * s**-1
466.434600791 m * s**-1
143546.758496 m * s**-1
488.077140148 m * s**-1
498.364885138 m * s**-1
542.157731738 m * s**-1
100102.189072 m * s**-1
761.901310115 m * s**-1
874.339707734 m * s**-1
1037.77328707 m * s**-1
8020.30885016 m * s**-1
1282.61593471 m * s**-1
859.778398679 m * s**-1
726.4318894 m * s**-1
616.06043773 m * s**-1
734.079546946 m * s**-1
559.235185445 m * s**-1
1337.69993184 m * s**-1
714.577545377 m * s**-1
4475.35883375 m * s**-1
2537.0984961 m * s**-1
1577.7386638 m * s**-1
1035.16585309 m * s**-1
50266.1196528 m * s**-1
8417.66883997 m * s**-1
2098.86061628 m * s**-1
527.148455474 m * s**-1
919.926622044 m * s**-1
921.478483186 m * s**-1
921.896077447 m * s**-1
921.983260984 m * s**-1
922.010420117 m * s**-1
3181.23583326 m * s**-1
1773.94676491 m * s**-1
15354.3723517 m * s**-1
1040099.03423 m * s**-

In [7]:
def read_initial_state(initial_dir, file_prefix):
    ''' Reads in an initial state for the Tycho Module.
        file_prefix: String Value for a Prefix to the Saved File
    '''
# TODO: Convert the saved datasets from SI to NBody. Also everything else in this function.

# First, Define the Directory where Initial State is Stored
#    file_dir = os.getcwd()+"/InitialState"
    file_dir = initial_dir
    file_base = file_dir+"/"+file_prefix
# Second, Read the Master AMUSE Particle Set from a HDF5 File
    file_format = "hdf5"
    master_set = read_set_from_file(file_base+"_particles.hdf5", format=file_format, close_file=True)
# Third, unPickle the Initial Conditions Array
    ic_file = open(file_base+"_ic.pkl", "rb")
    ic_array = pickle.load(ic_file)
    ic_file.close()
# Fourth, convert ic_array.total_smass and viral_radius from strings to floats
    total_smass = float(ic_array.total_smass) | units.kg
    viral_radius = float(ic_array.viral_radius) | units.m
# Fifth, Define the Master Set's Converter
    converter = nbody_system.nbody_to_si(total_smass, viral_radius)
    return master_set, ic_array, converter